# Handling Large Data

## Why Large Data Matters

In biomedical research, we routinely encounter datasets that challenge the limits of standard computing resources. Genomic studies may involve millions of genetic variants across thousands of samples. Single-cell RNA sequencing can produce expression measurements for 30,000 genes across hundreds of thousands of cells. Electronic health records for large hospital systems contain millions of patient encounters with dozens of variables each.

When your data doesn't fit comfortably in memory, naive approaches fail:
```python
import pandas as pd

# This might crash your computer if the file is large enough
df = pd.read_csv("massive_patient_records.csv")  # 50 GB file
```

The error message you'll see:

```
MemoryError: Unable to allocate 47.5 GiB for an array with shape (1000000000,) and data type float64
```

This lecture covers practical strategies for working with large datasets in Python: optimizing memory usage, processing data in chunks, using efficient file formats, and leveraging specialized data structures.

Before starting, let's first create the example data files that will be used.

In [ ]:
# Install required packages
%pip install pandas numpy scipy h5py pyarrow


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Setup: Create example data files used throughout this notebook
import pandas as pd
import numpy as np
import os

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

np.random.seed(42)

# Create patient_records.csv (used in pandas optimization examples)
n_patients = 100000
patients_df = pd.DataFrame({
    "patient_id": range(n_patients),
    "age": np.random.randint(18, 90, n_patients),
    "systolic_bp": np.random.normal(120, 15, n_patients),
    "diagnosis": np.random.choice(["healthy", "diabetes", "hypertension"], n_patients),
    "hospital": np.random.choice(["UNC", "Duke", "Wake Forest", "ECU"], n_patients)
})
patients_df.to_csv("data/patient_records.csv", index=False)

# Create large_patient_file.csv (used in chunking examples)
patients_df.to_csv("data/large_patient_file.csv", index=False)

# Create raw_patients.csv (used in transform_and_save example)
patients_df.to_csv("data/raw_patients.csv", index=False)

# Create expression_matrix.csv (gene expression data)
n_genes = 1000  # Small for demo; real data would have ~20,000 genes
n_samples = 50  # Small for demo; real data would have hundreds
gene_symbols = [f"GENE_{i}" for i in range(n_genes)]
sample_cols = [f"sample_{i:03d}" for i in range(n_samples)]

expression_data = {"gene_symbol": gene_symbols}
for col in sample_cols:
    expression_data[col] = np.random.randint(0, 50000, n_genes).astype(float)
expression_df = pd.DataFrame(expression_data)
expression_df.to_csv("data/expression_matrix.csv", index=False)

# Also save as parquet for parquet examples
expression_df.to_parquet("data/expression_matrix.parquet")

print("Created example files in data/:")
for f in ["patient_records.csv", "large_patient_file.csv", "raw_patients.csv",
          "expression_matrix.csv", "expression_matrix.parquet"]:
    size_kb = os.path.getsize(f"data/{f}") / 1024
    print(f"  {f}: {size_kb:.1f} KB")

Created example files in data/:
  patient_records.csv: 4282.5 KB
  large_patient_file.csv: 4282.5 KB
  raw_patients.csv: 4282.5 KB
  expression_matrix.csv: 389.0 KB
  expression_matrix.parquet: 334.2 KB


### Understanding Memory Usage

Before optimizing, we need to measure. Python provides several tools for understanding memory consumption.

The `sys.getsizeof()` function returns the size of an object in bytes:

In [ ]:
import sys

x = 42
print(f"Integer: {sys.getsizeof(x)} bytes")  # 28 bytes

y = 3.14159
print(f"Float: {sys.getsizeof(y)} bytes")  # 24 bytes

s = "hello"
print(f"String: {sys.getsizeof(s)} bytes")  # 54 bytes

# A list has overhead beyond its elements
numbers = [1, 2, 3, 4, 5]
print(f"List of 5 ints: {sys.getsizeof(numbers)} bytes")  # 104 bytes

Integer: 28 bytes
Float: 24 bytes
String: 46 bytes
List of 5 ints: 104 bytes


For pandas DataFrames, use the `memory_usage()` method:

In [ ]:
import pandas as pd
import numpy as np

# Create a sample clinical dataset
n_patients = 100000
df = pd.DataFrame({
    "patient_id": range(n_patients),
    "age": np.random.randint(18, 90, n_patients),
    "systolic_bp": np.random.normal(120, 15, n_patients),
    "diagnosis": np.random.choice(["healthy", "diabetes", "hypertension"], n_patients),
    "hospital": np.random.choice(["UNC", "Duke", "Wake Forest", "ECU"], n_patients)
})

# Check memory usage by column
print(df.memory_usage(deep=True))

Index              132
patient_id      800000
age             800000
systolic_bp     800000
diagnosis      5800691
hospital       5425173
dtype: int64


Output:
```
Index           128
patient_id    800000
age           800000
systolic_bp   800000
diagnosis    6400000
hospital     6400000
dtype: int64
```

The `deep=True` parameter is important for accurate measurement of string columns. Notice how the string columns (`diagnosis` and `hospital`) use 8x more memory than the numeric columns, even though they have only a few unique values.

In [ ]:
# Total memory in megabytes
total_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"Total memory: {total_mb:.2f} MB")

Total memory: 12.99 MB


### The Memory Problem with Default Data Types

Pandas uses memory-inefficient defaults. Let's examine why:

In [ ]:
print(df.dtypes)

patient_id       int64
age              int64
systolic_bp    float64
diagnosis       object
hospital        object
dtype: object


Output:
```
patient_id       int64
age              int64
systolic_bp    float64
diagnosis       object
hospital        object
dtype: object
```

The `int64` type uses 8 bytes per value, even for columns like `age` that only need values 0-120. The `object` type for strings stores each string as a separate Python object with significant overhead.

## Pandas Memory Optimization

The most impactful optimization is choosing appropriate data types. This section covers techniques that can reduce memory usage by 50-90%.

### Numeric Type Downcasting

For integer columns, choose the smallest type that fits your data:

| Type    | Bytes | Range |
|---------|-------|-------|
| int8    | 1     | -128 to 127 |
| int16   | 2     | -32,768 to 32,767 |
| int32   | 4     | -2.1 billion to 2.1 billion |
| int64   | 8     | -9.2 quintillion to 9.2 quintillion |
| uint8   | 1     | 0 to 255 |
| uint16  | 2     | 0 to 65,535 |

For floating-point columns:

| Type    | Bytes | Precision |
|---------|-------|-----------|
| float16 | 2     | ~3 decimal digits |
| float32 | 4     | ~7 decimal digits |
| float64 | 8     | ~15 decimal digits |

Let's optimize our clinical dataset:

In [ ]:
# Before optimization
print(f"Before: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Downcast integers
df["patient_id"] = pd.to_numeric(df["patient_id"], downcast="unsigned")
df["age"] = pd.to_numeric(df["age"], downcast="unsigned")

# Downcast floats (be careful with precision requirements)
df["systolic_bp"] = pd.to_numeric(df["systolic_bp"], downcast="float")

print(df.dtypes)

Before: 12.99 MB
patient_id      uint32
age              uint8
systolic_bp    float32
diagnosis       object
hospital        object
dtype: object


Output:
```
patient_id      uint32
age              uint8
systolic_bp    float32
diagnosis       object
hospital        object
dtype: object
```

The `age` column now uses 1 byte instead of 8, and `systolic_bp` uses 4 bytes instead of 8.

### The Categorical Data Type

For string columns with few unique values, the `category` dtype provides dramatic savings:

In [ ]:
# Check unique values
print(f"Unique diagnoses: {df['diagnosis'].nunique()}")  # 3
print(f"Unique hospitals: {df['hospital'].nunique()}")   # 4

# Convert to categorical
df["diagnosis"] = df["diagnosis"].astype("category")
df["hospital"] = df["hospital"].astype("category")

print(f"After: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Unique diagnoses: 3
Unique hospitals: 4
After: 1.05 MB


The categorical type stores an integer code for each row plus a small lookup table of unique values. For 100,000 rows with only 3-4 unique values, this reduces memory from ~6 MB per column to ~100 KB.

### Specifying Types at Load Time

Rather than loading data and then converting, specify types when reading:

In [ ]:
# Define the optimal types
dtype_spec = {
    "patient_id": "uint32",
    "age": "uint8",
    "systolic_bp": "float32",
    "diagnosis": "category",
    "hospital": "category"
}

# Load with specified types
df = pd.read_csv("data/patient_records.csv", dtype=dtype_spec)

This avoids the memory spike from loading with default types first.

### Loading Only Needed Columns

If you only need a subset of columns, use `usecols`:

In [ ]:
# Only load the columns we need for analysis
df = pd.read_csv(
    "data/patient_records.csv",
    usecols=["patient_id", "age", "diagnosis"],
    dtype={"patient_id": "uint32", "age": "uint8", "diagnosis": "category"}
)

For files with many columns, this can dramatically reduce memory usage and load time.

### Question

Consider this code that loads a gene expression dataset:

In [ ]:
import pandas as pd

# Gene expression matrix: 20,000 genes x 500 samples
# Expression values range from 0 to ~50,000 (RNA-seq counts)
# Sample IDs are strings like "TCGA-AB-1234"
# Gene symbols are strings like "TP53", "BRCA1", etc.

df = pd.read_csv("data/expression_matrix.csv")
print(df.dtypes)

gene_symbol     object
sample_000     float64
sample_001     float64
sample_002     float64
sample_003     float64
sample_004     float64
sample_005     float64
sample_006     float64
sample_007     float64
sample_008     float64
sample_009     float64
sample_010     float64
sample_011     float64
sample_012     float64
sample_013     float64
sample_014     float64
sample_015     float64
sample_016     float64
sample_017     float64
sample_018     float64
sample_019     float64
sample_020     float64
sample_021     float64
sample_022     float64
sample_023     float64
sample_024     float64
sample_025     float64
sample_026     float64
sample_027     float64
sample_028     float64
sample_029     float64
sample_030     float64
sample_031     float64
sample_032     float64
sample_033     float64
sample_034     float64
sample_035     float64
sample_036     float64
sample_037     float64
sample_038     float64
sample_039     float64
sample_040     float64
sample_041     float64
sample_042 

Output:
```
gene_symbol      object
sample_001      float64
sample_002      float64
...
sample_500      float64
```

The DataFrame uses 80 MB of memory. Describe three specific optimizations that could reduce memory usage and estimate the savings for each.

#### Answer

1. **Convert `gene_symbol` to categorical**: With ~20,000 unique gene symbols, categorical won't save much here since most values are unique. Skip this optimization for this column.

2. **Downcast expression values to float32**: RNA-seq counts don't need 15 decimal places of precision. Converting from float64 to float32 cuts memory in half for the numeric columns. Since expression values dominate the memory usage (500 columns x 20,000 rows x 8 bytes = 80 MB), this saves ~40 MB.

3. **Consider uint16 if counts are integers**: If the values are truly counts (integers 0-50,000), `uint16` (range 0-65,535) would use only 2 bytes per value instead of 8, saving 75% of numeric memory (~60 MB savings).

4. **Use `usecols` if analyzing a subset**: If only analyzing a subset of samples, loading only those columns avoids loading unnecessary data entirely.

Optimized loading:

In [ ]:
# If counts are integers 0-65535
df = pd.read_csv(
    "data/expression_matrix.csv",
    dtype={col: "uint16" for col in pd.read_csv("data/expression_matrix.csv", nrows=0).columns[1:]}
)

## Processing Data in Chunks

When data is too large to fit in memory even with optimizations, process it in chunks. This approach trades memory for processing time.

### The chunksize Parameter

Many pandas readers support the `chunksize` parameter, which returns an iterator instead of a DataFrame:

In [ ]:
import pandas as pd

# Process a large file 10,000 rows at a time
chunk_iter = pd.read_csv("data/large_patient_file.csv", chunksize=10000)

# Each iteration yields a DataFrame with 10,000 rows
for chunk in chunk_iter:
    print(f"Processing chunk with {len(chunk)} rows")
    # Process this chunk

Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows


### Aggregating Across Chunks

A common pattern is computing statistics across the entire dataset by combining partial results from each chunk:

In [ ]:
import pandas as pd

def compute_mean_age_by_diagnosis(filepath, chunksize=50000):
    """Compute mean age for each diagnosis across a large file."""
    # Accumulators
    sums = {}
    counts = {}

    for chunk in pd.read_csv(filepath, chunksize=chunksize):
        # Group by diagnosis within this chunk
        grouped = chunk.groupby("diagnosis")["age"]

        for diagnosis, group in grouped:
            if diagnosis not in sums:
                sums[diagnosis] = 0
                counts[diagnosis] = 0
            sums[diagnosis] += group.sum()
            counts[diagnosis] += len(group)

    # Compute final means
    return {diag: sums[diag] / counts[diag] for diag in sums}

result = compute_mean_age_by_diagnosis("data/large_patient_file.csv")
print(result)

{'diabetes': 53.32473124755337, 'healthy': 53.41256397210667, 'hypertension': 53.30394271675954}


### Chunked Processing with Generators

Generators provide a clean way to build streaming data pipelines:

In [ ]:
import pandas as pd

def read_and_filter(filepath, chunksize=50000):
    """Yield filtered chunks from a large CSV."""
    for chunk in pd.read_csv(filepath, chunksize=chunksize):
        # Filter to patients over 65
        filtered = chunk[chunk["age"] > 65]
        if len(filtered) > 0:
            yield filtered

def process_elderly_patients(filepath):
    """Process elderly patients without loading entire file."""
    total_patients = 0
    total_bp_sum = 0

    for chunk in read_and_filter(filepath):
        total_patients += len(chunk)
        total_bp_sum += chunk["systolic_bp"].sum()

    if total_patients > 0:
        return total_bp_sum / total_patients
    return None

mean_bp = process_elderly_patients("data/large_patient_file.csv")
print(f"Mean systolic BP for patients over 65: {mean_bp:.1f}")

Mean systolic BP for patients over 65: 120.0


### Writing Results in Chunks

When your output is also large, write incrementally:

In [ ]:
import pandas as pd

def transform_and_save(input_path, output_path, chunksize=50000):
    """Transform a large file and save results incrementally."""
    first_chunk = True

    for chunk in pd.read_csv(input_path, chunksize=chunksize):
        # Apply transformations
        chunk["age_group"] = pd.cut(chunk["age"], bins=[0, 18, 40, 65, 100],
                                    labels=["child", "young_adult", "middle_age", "senior"])
        chunk["bp_category"] = pd.cut(chunk["systolic_bp"], bins=[0, 120, 140, 200],
                                      labels=["normal", "elevated", "high"])

        # Write to output file
        chunk.to_csv(
            output_path,
            mode="w" if first_chunk else "a",
            header=first_chunk,
            index=False
        )
        first_chunk = False

transform_and_save("data/raw_patients.csv", "data/processed_patients.csv")

### Question

You have a 50 GB CSV file containing clinical trial data with columns: `patient_id`, `visit_date`, `measurement_type`, `value`. You need to compute the mean value for each measurement type. The file has 500 million rows and 100 unique measurement types.

Describe how you would approach this task using chunked processing. What data structures would you use to accumulate results? How would you compute the final means?

#### Answer

The approach uses two accumulators per measurement type: running sums and counts.

In [ ]:
import pandas as pd
from collections import defaultdict

def compute_measurement_means(filepath, chunksize=100000):
    # Accumulators: sum and count for each measurement type
    sums = defaultdict(float)
    counts = defaultdict(int)

    for chunk in pd.read_csv(filepath, chunksize=chunksize,
                             usecols=["measurement_type", "value"],
                             dtype={"measurement_type": "category", "value": "float32"}):
        # Aggregate within chunk
        grouped = chunk.groupby("measurement_type")["value"]

        for mtype, values in grouped:
            sums[mtype] += values.sum()
            counts[mtype] += len(values)

    # Compute final means
    return {mtype: sums[mtype] / counts[mtype] for mtype in sums}

Key points:
- Use `usecols` to only load needed columns (ignore `patient_id`, `visit_date`)
- Use `category` dtype for measurement_type (100 unique values)
- Use `float32` instead of `float64` to reduce memory
- `defaultdict` avoids key existence checks
- Only 100 measurement types means accumulators use negligible memory
- Each chunk processes independently, memory stays constant regardless of file size

## Efficient File Formats

CSV is human-readable but inefficient for large data. Binary formats offer significant improvements in size, speed, and functionality.

### CSV Limitations

CSV files have several drawbacks for large data:

- **No type information**: All data is stored as text and must be parsed
- **No compression**: Files are larger than necessary
- **Row-oriented**: Reading a single column requires scanning the entire file
- **Slow parsing**: Text parsing is computationally expensive

### Parquet: Columnar Storage

Parquet is a columnar format designed for analytics. It's the standard for big data ecosystems.

In [ ]:
import pandas as pd

# Write to Parquet (requires pyarrow: pip install pyarrow)
df.to_parquet("data/patients.parquet")

# Read from Parquet
df = pd.read_parquet("data/patients.parquet")

Parquet advantages:

- **Columnar storage**: Reading a subset of columns is fast
- **Built-in compression**: Typically 2-10x smaller than CSV
- **Type preservation**: Data types are stored in the file
- **Fast reads**: No text parsing needed

In [ ]:
import pandas as pd
import os

# Create a sample DataFrame
n = 1000000
df = pd.DataFrame({
    "patient_id": range(n),
    "age": [45] * n,
    "diagnosis": ["diabetes"] * n
})

# Compare file sizes
df.to_csv("data/test.csv", index=False)
df.to_parquet("data/test.parquet")

csv_size = os.path.getsize("data/test.csv") / 1024**2
parquet_size = os.path.getsize("data/test.parquet") / 1024**2
print(f"CSV: {csv_size:.2f} MB")
print(f"Parquet: {parquet_size:.2f} MB")
print(f"Compression ratio: {csv_size/parquet_size:.1f}x")

CSV: 18.01 MB
Parquet: 4.09 MB
Compression ratio: 4.4x


### Reading Column Subsets from Parquet

One major advantage of columnar storage is efficient column selection:

In [ ]:
import pandas as pd

# Only read two columns from a file with many columns
df = pd.read_parquet("data/expression_matrix.parquet", columns=["gene_symbol", "sample_001"])

This reads only the requested columns from disk, unlike CSV where the entire row must be scanned.

### Compression Options

Parquet supports several compression algorithms:

In [ ]:
# Default: snappy (fast, moderate compression)
df.to_parquet("data/data_snappy.parquet", compression="snappy")

# Best compression ratio
df.to_parquet("data/data_gzip.parquet", compression="gzip")

# Fast compression
df.to_parquet("data/data_lz4.parquet", compression="lz4")

# No compression
df.to_parquet("data/data_none.parquet", compression=None)

Choose based on your needs:
- **snappy**: Good default, balanced speed and size
- **gzip**: Smallest files, slower read/write
- **lz4**: Fastest, larger files than snappy

### Question

You're designing storage for a gene expression study with:
- 50,000 genes (rows)
- 10,000 samples (columns)
- Expression values are floats
- You frequently need to access all genes for a single sample
- You occasionally need to access a single gene across all samples
- Storage space is limited

What file format and layout would you choose? How would you organize the data?

#### Answer

This requires understanding how columnar vs row-oriented storage works:

**Columnar storage (Parquet)** stores each column contiguously on disk. Reading one column is fast (single contiguous read), but reading one row requires accessing every column file.

**Row-oriented storage (CSV)** stores each row contiguously. Reading one row is fast, but reading one column requires scanning the entire file.

Given the access patterns:
- **Frequent**: all genes for one sample → need to read one "sample column"
- **Occasional**: one gene across all samples → need to read one "gene row"

**Recommendation: Parquet with samples as columns** (genes as rows)

In this layout:
- Each sample is one column → reading all 50,000 genes for sample_001 reads one contiguous column = **fast**
- Each gene is one row → reading TP53 across all 10,000 samples requires accessing all column files = **slower, but acceptable for occasional use**

In [ ]:
# Organize as: rows = genes, columns = samples
expression_df.to_parquet(
    "data/expression.parquet",
    compression="snappy"  # Good balance of speed and size
)

# Fast: get all genes for sample_001
sample_data = pd.read_parquet("data/expression.parquet", columns=["gene_symbol", "sample_001"])

# Slower but possible: get one gene across all samples
all_data = pd.read_parquet("data/expression.parquet")
gene_data = all_data[all_data["gene_symbol"] == "TP53"]

For truly large datasets where both access patterns matter, consider HDF5 with chunking aligned to your access patterns, or storing two versions (transposed) if storage permits.

## HDF5 with h5py

HDF5 (Hierarchical Data Format version 5) is designed for storing and managing large amounts of scientific data. It's particularly useful for multi-dimensional arrays and complex data hierarchies.

### Why HDF5?

HDF5 offers several features valuable for scientific computing:

- **Hierarchical structure**: Organize data like a filesystem within a single file
- **Partial I/O**: Read slices of large arrays without loading everything
- **Compression**: Built-in support for various compression algorithms
- **Self-describing**: Metadata stored alongside data
- **Language-agnostic**: Files readable by R, MATLAB, Julia, etc.

### Creating HDF5 Files

The `h5py` library provides a Pythonic interface to HDF5:

In [ ]:
import h5py
import numpy as np

# Define dimensions as variables for flexibility
n_genes = 20000
n_samples = 500

# Create a new HDF5 file
with h5py.File("data/study_data.h5", "w") as f:
    # Create a dataset
    expression = np.random.rand(n_genes, n_samples).astype(np.float32)
    f.create_dataset("expression", data=expression)

    # Create a group (like a folder)
    metadata = f.create_group("metadata")

    # Add datasets to the group
    gene_names = np.array([f"GENE_{i}" for i in range(n_genes)], dtype="S20")
    metadata.create_dataset("gene_names", data=gene_names)

    sample_ids = np.array([f"SAMPLE_{i}" for i in range(n_samples)], dtype="S20")
    metadata.create_dataset("sample_ids", data=sample_ids)

    # Add attributes (metadata)
    f.attrs["study_name"] = "TCGA Breast Cancer"
    f.attrs["creation_date"] = "2024-01-15"
    f.attrs["n_genes"] = n_genes
    f.attrs["n_samples"] = n_samples

### Reading HDF5 Files

In [ ]:
import h5py

with h5py.File("data/study_data.h5", "r") as f:
    # List contents (like ls)
    print("Contents:", list(f.keys()))

    # Access attributes
    print("Study:", f.attrs["study_name"])

    # Read entire dataset
    expression = f["expression"][:]
    print("Shape:", expression.shape)

    # Read from nested groups
    gene_names = f["metadata/gene_names"][:]

Contents: ['expression', 'metadata']
Study: TCGA Breast Cancer
Shape: (20000, 500)


### Partial Reads: The Key Advantage

HDF5 allows reading slices without loading the entire dataset:

In [ ]:
import h5py

with h5py.File("data/study_data.h5", "r") as f:
    # Read only first 100 genes for first 10 samples
    # This reads only this slice from disk, not the full array
    subset = f["expression"][:100, :10]
    print("Subset shape:", subset.shape)

    # Read a single gene across all samples
    gene_42 = f["expression"][42, :]

    # Read expression for genes 1000-2000, samples 100-200
    region = f["expression"][1000:2000, 100:200]

Subset shape: (100, 10)


This is possible because HDF5 stores data in chunks and tracks chunk locations.

### Chunking for Efficient Access

Chunking divides the dataset into fixed-size blocks. Choose chunk shape based on access patterns:

In [ ]:
import h5py
import numpy as np

with h5py.File("data/chunked_data.h5", "w") as f:
    # Create dataset with explicit chunking
    # If we typically access rows (genes), chunk by rows
    f.create_dataset(
        "expression",
        shape=(20000, 500),
        dtype="float32",
        chunks=(100, 500)  # Each chunk is 100 genes x all samples
    )

    # Fill with data
    data = np.random.rand(20000, 500).astype(np.float32)
    f["expression"][:] = data

Access patterns and chunking:
- Access full rows frequently: `chunks=(small, full_width)`
- Access full columns frequently: `chunks=(full_height, small)`
- Random access: `chunks=(moderate, moderate)`

### Compression in HDF5

In [ ]:
import h5py
import numpy as np

with h5py.File("data/compressed_data.h5", "w") as f:
    data = np.random.rand(20000, 500).astype(np.float32)

    # gzip compression (most portable)
    f.create_dataset(
        "expression_gzip",
        data=data,
        compression="gzip",
        compression_opts=4  # 0-9, higher = more compression
    )

    # lzf compression (faster, less compression)
    f.create_dataset(
        "expression_lzf",
        data=data,
        compression="lzf"
    )

### Resizable Datasets

When you don't know the final size in advance:

In [ ]:
import h5py
import numpy as np

with h5py.File("data/growing_data.h5", "w") as f:
    # Create resizable dataset
    # maxshape=None means unlimited size in that dimension
    f.create_dataset(
        "measurements",
        shape=(0, 10),  # Start empty
        maxshape=(None, 10),  # Can grow in first dimension
        dtype="float32",
        chunks=(1000, 10)
    )

    # Add data in batches
    for batch in range(5):
        new_data = np.random.rand(1000, 10).astype(np.float32)

        # Resize and append
        current_size = f["measurements"].shape[0]
        f["measurements"].resize(current_size + 1000, axis=0)
        f["measurements"][current_size:current_size + 1000] = new_data

    print("Final shape:", f["measurements"].shape)  # (5000, 10)

Final shape: (5000, 10)


### Question

You're designing storage for a longitudinal multi-omics study with:
- 1000 patients
- 5 time points per patient
- 3 data types: genomics (1M variants), proteomics (5000 proteins), metabolomics (500 metabolites)
- Each measurement is a single float value

Sketch an HDF5 structure for this data. Consider: How would you organize the hierarchy? What chunking strategy would you use for each data type?

#### Answer

A hierarchical structure mirrors the study design:

```
study.h5
├── metadata/
│   ├── patient_ids          # (1000,) array of strings
│   ├── time_points          # (5,) array: [0, 6, 12, 18, 24] months
│   ├── variant_ids          # (1000000,) array of strings
│   ├── protein_ids          # (5000,) array of strings
│   └── metabolite_ids       # (500,) array of strings
│
├── genomics/                # 1M variants x 1000 patients x 5 timepoints
│   └── variants             # shape (1000000, 1000, 5), float32
│                            # chunks (10000, 100, 5) - access by variant
│
├── proteomics/              # 5000 proteins x 1000 patients x 5 timepoints
│   └── proteins             # shape (5000, 1000, 5), float32
│                            # chunks (500, 100, 5) - access by protein
│
└── metabolomics/            # 500 metabolites x 1000 patients x 5 timepoints
    └── metabolites          # shape (500, 1000, 5), float32
                             # chunks (500, 100, 5) - small enough for full feature access
```

Chunking rationale:
- **Genomics**: Large feature dimension, chunk to enable efficient access to subsets of variants
- **Proteomics/Metabolomics**: Smaller feature dimensions, can chunk more aggressively on patient dimension
- All include full time dimension in chunks since longitudinal analysis is common

## SQL Databases with SQLite

For tabular data with complex queries, relational databases offer advantages over flat files. SQLite is a lightweight database that requires no server setup.

### When to Use a Database

Databases excel when you need to:
- Query subsets of data without loading everything
- Join related tables
- Update or delete specific records
- Enforce data integrity (unique IDs, required fields)
- Share data with non-Python tools

### SQL Basics

SQL (Structured Query Language) is the standard language for interacting with relational databases. Here are the key commands we'll use:

**Creating tables:**
- `CREATE TABLE` defines a new table with column names and data types
- `PRIMARY KEY` uniquely identifies each row (like patient_id)
- `FOREIGN KEY` links to another table's primary key (for relationships)

**Data types in SQLite:**
- `TEXT` - strings
- `INTEGER` - whole numbers  
- `REAL` - floating-point numbers
- `BLOB` - binary data

### Creating a SQLite Database

In [ ]:
import sqlite3
import pandas as pd

# Create connection (creates file if it doesn't exist)
conn = sqlite3.connect("data/clinical_study.db")

# CREATE TABLE defines a new table
# Column format: column_name DATA_TYPE [constraints]
# PRIMARY KEY: ensures each patient_id is unique and not null
conn.execute("""
    CREATE TABLE IF NOT EXISTS patients (
        patient_id TEXT PRIMARY KEY,
        age INTEGER,
        sex TEXT,
        diagnosis TEXT,
        enrollment_date TEXT
    )
""")

# FOREIGN KEY links lab_results.patient_id to patients.patient_id
# This ensures we can't add lab results for non-existent patients
# AUTOINCREMENT automatically assigns increasing IDs to new rows
conn.execute("""
    CREATE TABLE IF NOT EXISTS lab_results (
        result_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        test_date TEXT,
        test_name TEXT,
        value REAL,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
""")

conn.commit()
conn.close()

### pandas Integration

pandas provides seamless reading and writing to SQL databases:

In [ ]:
import pandas as pd
import sqlite3
import numpy as np

# Create sample data
patients = pd.DataFrame({
    "patient_id": [f"P{i:04d}" for i in range(10000)],
    "age": np.random.randint(18, 90, 10000),
    "sex": np.random.choice(["M", "F"], 10000),
    "diagnosis": np.random.choice(["healthy", "diabetes", "cancer", "cardiovascular"], 10000),
    "enrollment_date": pd.date_range("2020-01-01", periods=10000, freq="h").astype(str)
})

# Write to database
conn = sqlite3.connect("data/clinical_study.db")
patients.to_sql("patients", conn, if_exists="replace", index=False)
conn.close()

### Querying with SQL

The power of databases is efficient querying. The basic query structure is:

```sql
SELECT columns FROM table WHERE condition
```

- `SELECT` specifies which columns to retrieve (`*` means all columns)
- `FROM` specifies the table
- `WHERE` filters rows based on conditions
- `GROUP BY` groups rows for aggregation
- `AVG()`, `COUNT()`, `SUM()` are aggregate functions

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/clinical_study.db")

# SELECT * returns all columns from the patients table
df = pd.read_sql("SELECT * FROM patients", conn)

# WHERE clause filters rows
# AND combines multiple conditions (both must be true)
elderly_diabetics = pd.read_sql("""
    SELECT * FROM patients
    WHERE age > 65 AND diagnosis = 'diabetes'
""", conn)

# GROUP BY groups rows with same diagnosis value
# Aggregate functions compute one value per group:
#   COUNT(*) counts rows in each group
#   AVG(age) computes mean age for each group
diagnosis_counts = pd.read_sql("""
    SELECT diagnosis, COUNT(*) as count, AVG(age) as mean_age
    FROM patients
    GROUP BY diagnosis
""", conn)

print(diagnosis_counts)
conn.close()

        diagnosis  count   mean_age
0          cancer   2593  54.303509
1  cardiovascular   2537  53.305085
2        diabetes   2407  53.288741
3         healthy   2463  53.691839


### Parameterized Queries

Never build SQL strings with f-strings or concatenation. This creates a security vulnerability called **SQL injection**, where malicious input can execute unintended commands.

For example, if a user enters `'; DROP TABLE patients; --` as their diagnosis, a naive query could delete your entire table!

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/clinical_study.db")

# Safe: parameterized query using ? placeholders
# The database driver escapes special characters in the parameters
min_age = 65
diagnosis = "diabetes"

df = pd.read_sql(
    "SELECT * FROM patients WHERE age > ? AND diagnosis = ?",
    conn,
    params=(min_age, diagnosis)
)

conn.close()

Always sanitize your inputs and use parameterized queries to prevent this!

![Bobby Tables](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

[xkcd.com/327](https://xkcd.com/327/)

### Joining Tables

`JOIN` combines rows from two tables based on a related column. This is one of the most powerful features of relational databases.

```sql
SELECT columns
FROM table1
JOIN table2 ON table1.key = table2.key
```

The `ON` clause specifies how to match rows between tables.

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/clinical_study.db")

# JOIN combines patients (p) with their lab_results (l)
# ON specifies the matching column between tables
# p. and l. are aliases for table names (shorter to type)
# This query finds HbA1c values for diabetic patients
query = """
    SELECT p.patient_id, p.age, p.diagnosis,
           l.test_name, l.value, l.test_date
    FROM patients p
    JOIN lab_results l ON p.patient_id = l.patient_id
    WHERE p.diagnosis = 'diabetes'
      AND l.test_name = 'HbA1c'
"""

diabetic_hba1c = pd.read_sql(query, conn)
conn.close()

### Indexing for Performance

For large tables, indexes dramatically speed up queries. An index is like a book's index: it lets the database quickly find rows matching a condition without scanning every row.

Create indexes on columns you frequently filter or join on:

In [ ]:
import sqlite3

conn = sqlite3.connect("data/clinical_study.db")

# CREATE INDEX creates a lookup structure for faster queries
# idx_diagnosis speeds up: WHERE diagnosis = '...'
# idx_patient_lab speeds up: JOIN ... ON patient_id = ...
conn.execute("CREATE INDEX IF NOT EXISTS idx_diagnosis ON patients(diagnosis)")
conn.execute("CREATE INDEX IF NOT EXISTS idx_patient_lab ON lab_results(patient_id)")

conn.commit()
conn.close()

### Question

You have a SQLite database with tables `patients` (patient_id, age, sex, site) and `visits` (visit_id, patient_id, visit_date, systolic_bp, diastolic_bp). Write a SQL query to find the mean systolic blood pressure for female patients over 60, grouped by site.

#### Answer

%%sql
SELECT p.site, AVG(v.systolic_bp) as mean_systolic_bp, COUNT(*) as n_visits
FROM patients p
JOIN visits v ON p.patient_id = v.patient_id
WHERE p.sex = 'F' AND p.age > 60
GROUP BY p.site
ORDER BY mean_systolic_bp DESC

In pandas:

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/clinical_study.db")
result = pd.read_sql("""
    SELECT p.site, AVG(v.systolic_bp) as mean_systolic_bp, COUNT(*) as n_visits
    FROM patients p
    JOIN visits v ON p.patient_id = v.patient_id
    WHERE p.sex = 'F' AND p.age > 60
    GROUP BY p.site
    ORDER BY mean_systolic_bp DESC
""", conn)
conn.close()

DatabaseError: Execution failed on sql '
    SELECT p.site, AVG(v.systolic_bp) as mean_systolic_bp, COUNT(*) as n_visits
    FROM patients p
    JOIN visits v ON p.patient_id = v.patient_id
    WHERE p.sex = 'F' AND p.age > 60
    GROUP BY p.site
    ORDER BY mean_systolic_bp DESC
': no such table: visits

## Sparse Matrices with scipy.sparse

Many biomedical datasets are sparse: most values are zero. Examples include:
- Single-cell RNA-seq: Most genes have zero counts in most cells
- Genetic variants: Most individuals have the reference allele at most positions
- Document-term matrices: Most documents don't contain most words
- Network adjacency matrices: Most nodes aren't connected

### The Memory Problem with Dense Storage

Consider a single-cell gene expression matrix:

In [ ]:
import numpy as np

n_genes = 30000
n_cells = 100000

# Dense storage (even if 95% zeros)
# Memory: 30000 * 100000 * 8 bytes = 24 GB
dense_matrix = np.zeros((n_genes, n_cells), dtype=np.float64)

Even storing zeros uses 8 bytes each. For a 95% sparse matrix, we're wasting 95% of memory.

### Sparse Matrix Formats

scipy.sparse provides several formats optimized for different operations:

**COO (Coordinate)**: Stores (row, col, value) triplets
- Good for: Construction, converting to other formats
- Bad for: Arithmetic, slicing

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix

# Create from coordinate lists
rows = np.array([0, 0, 1, 2, 2])
cols = np.array([0, 2, 1, 0, 2])
data = np.array([1.0, 2.0, 3.0, 4.0, 5.0])

sparse_coo = coo_matrix((data, (rows, cols)), shape=(3, 3))
print(sparse_coo.toarray())

Output:
```
[[1. 0. 2.]
 [0. 3. 0.]
 [4. 0. 5.]]
```

**CSR (Compressed Sparse Row)**: Stores data row-by-row
- Good for: Row slicing, matrix-vector products
- Standard format for machine learning

CSR uses three arrays:
- `data`: non-zero values, stored row-by-row: `[1, 2, 3, 4, 5]`
- `indices`: column index of each value: `[0, 2, 1, 0, 2]`
- `indptr`: where each row starts in data array: `[0, 2, 3, 5]`
  - Row 0 spans data[0:2], Row 1 spans data[2:3], Row 2 spans data[3:5]

In [ ]:
from scipy.sparse import csr_matrix

sparse_csr = coo_matrix((data, (rows, cols)), shape=(3, 3)).tocsr()

# Efficient row access
row_1 = sparse_csr[1, :]  # Fast
print("Row 1:", row_1.toarray())

**CSC (Compressed Sparse Column)**: Stores data column-by-column
- Good for: Column slicing, solving linear systems

CSC is like CSR but organized by columns instead of rows:
- `data`: non-zero values, stored column-by-column
- `indices`: row index of each value
- `indptr`: where each column starts in data array

In [ ]:
from scipy.sparse import csc_matrix

sparse_csc = coo_matrix((data, (rows, cols)), shape=(3, 3)).tocsc()

# Efficient column access
col_2 = sparse_csc[:, 2]  # Fast
print("Col 2:", col_2.toarray())

### Memory Comparison

Let's compare the memory usage of dense vs sparse storage:

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import sys

# Create a sparse matrix (95% zeros)
n = 10000
density = 0.05  # 5% non-zero

np.random.seed(42)
dense = np.zeros((n, n))
n_nonzero = int(n * n * density)
rows = np.random.randint(0, n, n_nonzero)
cols = np.random.randint(0, n, n_nonzero)
dense[rows, cols] = np.random.rand(n_nonzero)

sparse = csr_matrix(dense)

# Compare memory
dense_mb = dense.nbytes / 1024**2
sparse_mb = (sparse.data.nbytes + sparse.indices.nbytes + sparse.indptr.nbytes) / 1024**2

print(f"Dense: {dense_mb:.1f} MB")
print(f"Sparse: {sparse_mb:.1f} MB")
print(f"Ratio: {dense_mb/sparse_mb:.1f}x savings")

Output:
```
Dense: 762.9 MB
Sparse: 57.2 MB
Ratio: 13.3x savings
```

### Operations on Sparse Matrices

Most numpy-like operations work on sparse matrices:

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, random

# Create two sparse matrices
A = random(1000, 1000, density=0.01, format="csr")
B = random(1000, 1000, density=0.01, format="csr")

# Matrix multiplication (stays sparse)
C = A @ B

# Element-wise operations
D = A + B
E = A.multiply(B)  # Element-wise, not @

# Scalar operations
F = A * 2.0

# Statistics (may convert to dense internally for some operations)
row_sums = np.array(A.sum(axis=1)).flatten()
col_means = np.array(A.mean(axis=0)).flatten()

### Converting Between Formats

In [ ]:
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix

# Start with COO (good for construction)
coo = coo_matrix((data, (rows, cols)), shape=(1000, 1000))

# Convert for row operations
csr = coo.tocsr()

# Convert for column operations
csc = coo.tocsc()

# Convert back to dense (careful with memory!)
dense = csr.toarray()  # Only if matrix is small enough

### Sparse Matrices in Practice: Single-Cell Data

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, save_npz, load_npz
from scipy.sparse import random

# Simulate single-cell expression (highly sparse)
n_genes = 20000
n_cells = 50000
density = 0.05  # 5% of gene-cell pairs have non-zero expression

# Generate sparse data directly
expression = random(n_genes, n_cells, density=density,
                    format="csr", dtype=np.float32)

# Save sparse matrix
save_npz("data/expression_sparse.npz", expression)

# Load sparse matrix
expression_loaded = load_npz("data/expression_sparse.npz")

# Compute gene statistics efficiently
gene_means = np.array(expression.mean(axis=1)).flatten()
gene_nonzero_counts = np.diff(expression.indptr)  # CSR gives this for free

print(f"Shape: {expression.shape}")
print(f"Non-zero elements: {expression.nnz:,}")
print(f"Sparsity: {1 - expression.nnz / (n_genes * n_cells):.1%}")

### Question

You have a gene-cell expression matrix where 97% of values are zero. The matrix has 25,000 genes and 200,000 cells. Expression values are counts ranging from 0 to 1000.

1. Estimate the memory for dense vs sparse storage
2. What sparse format would you use and why?
3. What data type would you use for the values?

#### Answer

1. **Memory estimation**:
   - Dense: 25,000 × 200,000 × 8 bytes (float64) = 40 GB
   - Sparse (3% non-zero): ~150 million non-zero values
     - CSR: data (150M × 4 bytes) + indices (150M × 4 bytes) + indptr (25K × 4 bytes) ≈ 1.2 GB
   - Savings: ~33x

2. **Format choice**: **CSR** (Compressed Sparse Row)
   - Gene expression analysis typically computes statistics per gene (row-wise operations)
   - CSR is efficient for row slicing and row-wise aggregations
   - Also efficient for matrix-vector products common in dimensionality reduction

3. **Data type**: **uint16** or **float32**
   - Counts 0-1000 fit in uint16 (0-65535), using 2 bytes instead of 8
   - If normalized values are needed later, float32 (4 bytes) provides sufficient precision
   - With uint16: sparse storage drops to ~750 MB

## Brief Survey of Alternatives

When pandas becomes insufficient, several libraries provide scaled-up alternatives.

### Dask: Parallel pandas

Dask extends pandas to larger-than-memory datasets by breaking them into partitions:

In [ ]:
import dask.dataframe as dd

# Read large CSV as Dask DataFrame
ddf = dd.read_csv("data/large_patient_file.csv")

# Familiar pandas-like API
result = ddf.groupby("diagnosis")["age"].mean()

# Computation is lazy - nothing happens until .compute()
print(result.compute())

Dask is useful when:
- Data is larger than memory but fits on disk
- Operations can be parallelized across partitions
- You want to stay close to pandas syntax

### Polars: Fast DataFrames

Polars is a newer library written in Rust, offering significant speed improvements:

In [ ]:
import polars as pl

# Read CSV
df = pl.read_csv("data/patient_records.csv")

# Eager execution (like pandas)
result = df.filter(pl.col("age") > 65).group_by("diagnosis").agg(
    pl.col("systolic_bp").mean()
)

# Lazy execution (optimized query planning)
ldf = pl.scan_csv("data/patient_records.csv")  # Doesn't load data yet
result = (
    ldf.filter(pl.col("age") > 65)
    .group_by("diagnosis")
    .agg(pl.col("systolic_bp").mean())
    .collect()  # Executes optimized query
)

Polars excels at:
- Pure speed (often 10-100x faster than pandas)
- Memory efficiency
- Query optimization through lazy evaluation

### Choosing the Right Tool

| Situation | Recommended Tool |
|-----------|------------------|
| Data fits in memory, complex analysis | pandas (optimized dtypes) |
| Data larger than memory, pandas-like workflow | Dask |
| Speed-critical analysis | Polars |
| Hierarchical scientific data, partial reads | HDF5 |
| Relational data, complex queries | SQL database |
| Highly sparse data | scipy.sparse |

For most biomedical research, optimized pandas with efficient file formats (Parquet, HDF5) handles the majority of use cases. Move to specialized tools when you hit specific limitations.

## References and Further Reading

**pandas**
- [Scaling to Large Datasets](https://pandas.pydata.org/docs/user_guide/scale.html) - Official pandas documentation on memory optimization and chunking

**HDF5**
- [h5py Documentation](https://docs.h5py.org/) - Python interface to HDF5
- [HDF5 User Guide](https://portal.hdfgroup.org/documentation/) - Comprehensive HDF5 documentation

**Sparse Matrices**
- [scipy.sparse Tutorial](https://docs.scipy.org/doc/scipy/tutorial/sparse.html) - Official SciPy sparse matrix documentation

**Alternative Libraries**
- [Dask Documentation](https://docs.dask.org/) - Parallel computing with pandas-like API
- [Polars User Guide](https://docs.pola.rs/) - Fast DataFrame library

**SQL**
- [SQLite Tutorial](https://www.sqlitetutorial.net/) - Learn SQL with SQLite
- [pandas SQL Documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) - pandas-SQL integration